In [ ]:
#setup

In [5]:
import sys, os
sys.path.insert(0,os.path.realpath("./fastai/old"))

import fastai

from fastai.text import *

import warnings

#from fastai.plots import *

from sklearn.metrics import confusion_matrix
import functools

from scipy.stats import zscore

import requests
from io import BytesIO
from zipfile import ZipFile
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import datetime as dt
import matplotlib.dates as mdates
from datetime import datetime, timedelta

In [4]:
torch.backends.cudnn.enabled = False


In [ ]:
PATH=Path('data/boc/')

CLAS_PATH=Path('data/boc_clas/')
CLAS_PATH.mkdir(exist_ok=True)

LM_PATH=Path('data/boc_lm/')
LM_PATH.mkdir(exist_ok=True)